In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:8 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1,012 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,497 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:13 http://ar

In [2]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-02 16:22:30--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2023-03-02 16:22:30 (10.5 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [4]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
shoes = 'https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz'
spark.sparkContext.addFile(shoes)

shoes_df = spark.read.csv(SparkFiles.get('amazon_reviews_us_Shoes_v1_00.tsv.gz'), sep='\t', header=True, inferSchema=True)

shoes_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [5]:
# Get the number of rows in the DataFrame.
shoes_df.count()

4366916

# Transform the Data

## Create the "review_id_table".

In [6]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = shoes_df.select('review_id', 'customer_id', 'product_id', 'product_parent', to_date('review_date', 'yyy-mm-dd')).withColumnRenamed('to_date(review_date, yyy-mm-dd)', 'review_date')

review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781| 2015-01-31|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853| 2015-01-31|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660| 2015-01-31|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263| 2015-01-31|
|R2EQ1TG9IT3OEQ|   45449350|B00SW64Y9W|       7853171| 2015-01-31|
|R1WXA9JSC2H1U1|   19324665|B011F9E6LI|      14311457| 2015-01-31|
|R12ENYLFGGNWRV|   50073594|B00HAUP1OI|     264821602| 2015-01-31|
|R2R07E5PNXEUO5|   21706057|B00L1RKOKW|     767118055| 2015-01-31|
|R27BA52AKWMWN3|   13708216|B005WA9MSE|     813856438| 2015-01-31|
| RLF8DOID2KD5O|   40542649|B00BEYQI5C|     661491213| 2015-01-31|
|R369CEXHXHC6NQ|   13409446|B00EYAFTCQ|     332158165| 2015-01-31|
|R171PPIJXFONVI|    9451727|B00I0QHY32|      49243908| 2015-01

## Create the "products" Table

In [7]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = shoes_df.select('product_id', 'product_title').dropDuplicates(['product_id', 'product_title'])

if products_df.count() > products_df.dropDuplicates(['product_id', 'product_title']).count():
    raise ValueError('Data has duplicates')
else:
  print('Data does not have dupes')

Data does not have dupes


## Create the "customers" Table

In [8]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = shoes_df.groupBy('customer_id').count().withColumnRenamed('count', 'customer_count')

customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16121565|             5|
|   48146680|             1|
|   11062912|             1|
|   51451778|             1|
|   27920838|             1|
|    4919528|             3|
|   47802851|             2|
|   42560427|             1|
|   29467780|             1|
|   49703087|            12|
|   38456994|             4|
|     557836|             1|
|   43783459|             1|
|   23607725|             1|
|   45632184|             1|
|   19979069|             2|
|   28380959|             1|
|   46563381|             1|
|   21628739|             3|
|   23042837|             2|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [9]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = shoes_df.select('review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine').withColumn('star_rating', shoes_df.star_rating.cast('int'))

vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R3P2HIOQCIN5ZU|          1|            0|          0|   N|
|R12VVR0WH5Q24V|          5|            0|          0|   N|
| RNCCKB6TV5EEF|          4|            0|          0|   N|
|R2NZXYIVCGB13W|          5|            0|          6|   N|
|R2EQ1TG9IT3OEQ|          3|            0|          0|   N|
|R1WXA9JSC2H1U1|          5|            1|          1|   N|
|R12ENYLFGGNWRV|          5|            1|          1|   N|
|R2R07E5PNXEUO5|          4|            0|          0|   N|
|R27BA52AKWMWN3|          5|            0|          0|   N|
| RLF8DOID2KD5O|          3|            0|          0|   N|
|R369CEXHXHC6NQ|          5|            0|          0|   N|
|R171PPIJXFONVI|          5|            0|          0|   N|
|R2JDNM8F2P06FU|          4|            1|          1|   N|
|R2W977FO4M97XT|          5|            

# Load

In [10]:
mode = "append"
jdbc_url="jdbc:postgresql://database-1.cm22pgcosamb.us-east-1.rds.amazonaws.com:5432/big_data2_db"
config = {"user":"root", "password": "postgres", "driver":"org.postgresql.Driver"}

In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [12]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [13]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [14]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)